In [1]:
import ir_measures

In [2]:
# paths

baseline = '../data/generated_conversations/bm25_t5_monot5_bart/bm25_t5_monot5_bart.run'
prompted_llama = "../data/generated_conversations/prompted_simulation_baseline_v2/prompted_simulation_baseline_v2.run"
tuned_llama = "../data/generated_conversations/bm25_t5_bart_llama2_t5_bart/bm25_t5_bart_llama2_t5_bart.run"
prompted_openai = "../data/generated_conversations/gpt3_simulation_baseline/gpt3_simulation_baseline.run"

In [3]:
MRR = ir_measures.RR()
RECALL = ir_measures.R()@1000
NDCG_CUT_3 = ir_measures.nDCG@3
NDCG_CUT_1000 = ir_measures.nDCG@1000
MAP = ir_measures.AP()@1000

In [4]:
import csv

turns_to_score = []

with open("annotations.csv") as csvfile:
    reader = csv.reader(csvfile)
    for idx, line in enumerate(reader):
        if idx < 2:
            continue
        turn_id, prompted_llama_annotation, prompted_openai_annotation, tuned_llama_annotation = line
        # ["follow_up", "positive_feedback", "good_cq_answer"] -> follow_up
        # ["unrelated_feedback"] -> unrelated_feedback
        # ["negative_with_clarification"] -> negative_with_clarification
        # ["negative_feedback"] -> negative_feedback
        # ["abdication"] -> abdication
        if tuned_llama_annotation in ["abdication"]:
            turns_to_score.append(turn_id)
print(turns_to_score)

['20-2_2', '20-1_2', '18-2_4', '18-2_3', '18-1_15', '18-1_13', '18-1_10', '17-3_9', '17-3_14', '17-3_1', '17-2_10', '17-1_9', '17-1_1', '16-2_2', '16-1_7', '16-1_6', '15-2_3', '14-2_8', '14-1_2', '14-1_17', '11-2_2', '11-1_1', '10-3_4', '10-3_2', '10-2_7', '10-2_6', '10-2_3']


In [5]:
qrels = list(ir_measures.read_trec_qrels('../../data/datasets/ikat/2023-qrels.all-turns.txt'))
qrels = [q for q in qrels if q.query_id in turns_to_score]
run = ir_measures.read_trec_run(tuned_llama)

In [6]:
ir_measures.calc_aggregate([MRR, RECALL, NDCG_CUT_3, NDCG_CUT_1000, MAP], qrels, run)

{nDCG@1000: 0.13833351052250117,
 R@1000: 0.18101876121678104,
 nDCG@3: 0.1346389360644392,
 AP@1000: 0.05581918952150833,
 RR: 0.33650755374893304}